# Basic Functions

In [1]:
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib

In [2]:
def u(t, T, scale=5000):
    return scale * np.sin(2 * np.pi * t / T)

In [3]:
# nonlinearity
def phi(x, scale=10):
    return 1 / (1 + np.exp(np.maximum(-250, np.minimum(250, -scale * x))))


def f_I_matrix_multiply(w, x):
    return w @ x


def get_dx(X, w, i, f_I, phi, u, alpha, theta, t, delay_i, T, dt, stimulate=True):
    I_in = f_I(w, X[i - delay_i])
    return (-X[i - 1] + phi(int(stimulate) * u(t[i] + theta, T) + I_in)) * dt * alpha


def get_dw(X, dx, w, i, beta, lmbda, delay_i, dt):
    return (-lmbda * w + np.outer(dx / dt, X[i - delay_i])) * dt * beta  # post <- pre

In [4]:
def simulate(
    dt,
    N,
    T,
    t,
    theta,
    alpha,
    beta,
    delay_i,
    lmbda,
    w,
    get_dw,
    get_dx,
    f_I,
    phi,
    u,
    stimulate=True,
    full_info=True,
    init_stimulation=True,
    init_travel=1,
    init_steps=None
):
    N_STEPS = len(t)
    X = np.zeros((N_STEPS, N))
    W = [w.copy()]
    DW = []
    DX = []

    if init_steps is None:
        if init_travel == 0:
            init_steps = delay_i
            t_init = np.zeros(delay_i)
        else:
            init_steps = 0
            while init_steps <= delay_i:
                init_steps += abs(round(N / init_travel))
            if init_travel > 0:
                t_init = np.arange(0, init_steps, init_travel*dt)
            else:
                t_init = np.arange(init_steps, 0, init_travel*dt)
    else:
        if init_travel == 0:
            t_init = np.zeros(init_steps)
        elif init_travel > 0:
            t_init = np.arange(0, init_steps, init_travel*dt)
        else:
            t_init = np.arange(init_steps, 0, init_travel*dt)
        
    
    for i in range(1, init_steps):
        dx = get_dx(
            X, w, i, lambda w, x: 0, phi, u, alpha, theta, t_init,
            delay_i, T, dt, stimulate=init_stimulation
        )
        X[i] = X[i - 1] + dx
    
        if full_info:
            W.append(w.copy())
            DW.append(np.zeros_like(w))
            DX.append(dx.copy())

    for i in range(init_steps, N_STEPS):
        dx = get_dx(X, w, i, f_I, phi, u, alpha, theta, t, delay_i, T, dt, stimulate=stimulate)
        dw = get_dw(X, dx, w, i, beta, lmbda, delay_i, dt)

        # Update the neuronal activity
        X[i] = X[i - 1] + dx

        # Update the synaptic weights
        w += dw

        if full_info:
            W.append(w.copy())
            DW.append(dw.copy())
            DX.append(dx.copy())
    if full_info:
        return X, w, np.array(W), np.array(DW), np.array(DX)
    else:
        return X, w

# Constants

In [23]:
dt = 2e-4
N = 500  # number of neurons
T = N * dt  # period, in seconds

rolling_indices = np.array([np.roll(np.arange(N), -i) for i in range(N)])

simulation_duration = 150 * T
t = np.arange(0, simulation_duration, dt)

theta = np.linspace(0, T, N, endpoint=False)  # spatial variable for neurons

# time constants, in seconds
tau_x = 5 * 1e-3
alpha = 1 / tau_x

tau_d = 15 * 1e-3
delay_i = round(tau_d / dt)

tau_w = 50
beta = 1 / tau_w

lmbda = 20

# Full Connectivity Simulation

$$
\text{input current \hspace{0.2cm}} u(x, t) = u(x, t + T) = u(0, t+x)
$$

$$
\text{firing rate \hspace{0.1cm}} r(x, t)
$$
$$
\tau_r \frac{\partial}{\partial t} r(x, t) = -r(x, t) + \varphi\bigg(u(x, t) + \int_0^T w(x, y, t) r(y, t - \tau_d) dy\bigg)
$$

$$
\text{learned weights \hspace{0.1cm}} \lim_{t\to \infty} w(\Delta x, t)
$$

$$
\text{learned weights from $y$ to $x$ \hspace{0.1cm}} \lim_{t\to \infty} w(x, y, t)
$$
$$
\tau_w \frac{\partial}{\partial t} w(x, y, t) = {r}(y, t-\tau_d)\dot{r}(x, t) - \lambda {w}(x, y, t)
$$

In [6]:
w_full = 0.05 * np.random.randn(N, N)

X_full, w_full = simulate(
    dt,
    N,
    T,
    t,
    theta,
    alpha,
    beta,
    delay_i,
    lmbda,
    w_full,
    get_dw,
    get_dx,
    f_I_matrix_multiply,
    phi,
    u,
    full_info=False,
)

# Weight Symmetry

We observe a periodicity of network activity and synaptic weights.

In [7]:
def symmetrize(A, mod):
    return np.array(
        [np.roll(row, -round(N * i / mod % mod)) for i, row in enumerate(A)]
    )

In [8]:
def f_I_conv_circ(signal, ker):
    return np.array(
        [np.sum(signal * np.roll(ker[::-1], n)) for n in range(1, len(signal) + 1)]
    )

def f_I_conv_circ_fft(signal, ker):
    return np.real(np.fft.ifft(np.fft.fft(signal) * np.fft.fft(ker)))

In [16]:
w = w_full[:, 0]

In [9]:
def get_dw_prime(X, dx, w, i, beta, lmbda, delay_i, dt):
    return (-lmbda * w + X[i - delay_i, 0] * (dx / dt)) * dt * beta

In [10]:
def get_dw_prime_mean(X, dx, w, i, beta, lmbda, delay_i, dt):
    return (
        (
            -lmbda * w * dt
            + (dx[rolling_indices] * X[i - delay_i][:, None]).mean(0)
        )
        * beta
    )

In [24]:
w = 0.05 * np.random.randn(N)

X, w, W, DW, DX = simulate(
    dt,
    N,
    T,
    t,
    theta,
    alpha,
    beta,
    delay_i,
    lmbda,
    w,
    get_dw_prime_mean,
    get_dx,
    f_I_conv_circ_fft,
    phi,
    u,
    full_info=True,
)